In [1]:
import sys

In [2]:
# Aggiungi al sys.path
sys.path.append("../preprocessing")

In [3]:
import sampling as samp

In [4]:
df = samp.sampling_csv("../timeseries/B101/Channel1.csv", 4)

In [5]:
df

,singleData
0,-21702628.0
4,17132264.0
8,28665146.0
12,36802184.0
16,47670068.0
...,...
12287980,48777768.0
12287984,-10236032.0
12287988,-16138004.0
12287992,-25018178.0


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.utils import parallel_backend
from tqdm import tqdm

In [7]:
print(df.head())
print(df.info())  # Controlliamo il tipo di dati e se ci sono NaN
print(df.describe())  # Controlliamo media, deviazione standard, ecc.

    singleData
0  -21702628.0
4   17132264.0
8   28665146.0
12  36802184.0
16  47670068.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3072000 entries, 0 to 12287996
Data columns (total 1 columns):
 #   Column      Dtype  
---  ------      -----  
 0   singleData  float64
dtypes: float64(1)
memory usage: 23.4 MB
None
         singleData
count  3.072000e+06
mean  -8.819765e+04
std    4.077789e+07
min   -3.156354e+08
25%   -2.599870e+07
50%   -1.116589e+06
75%    2.460922e+07
max    3.357127e+08


In [8]:
df.reset_index(drop=True, inplace=True)

In [9]:
df

,singleData
0,-21702628.0
1,17132264.0
2,28665146.0
3,36802184.0
4,47670068.0
...,...
3071995,48777768.0
3071996,-10236032.0
3071997,-16138004.0
3071998,-25018178.0


In [10]:
n_lags = 5  # Usiamo i 5 valori precedenti per prevedere il successivo

# Creiamo le colonne con i lag
for i in range(1, n_lags + 1):
    df[f'lag_{i}'] = df['singleData'].shift(i)

# Rimuoviamo le righe con valori NaN dovuti allo shift
df.dropna(inplace=True)

# Separiamo feature (X) e target (y)
X = df.drop(columns=['singleData']).values
y = df['singleData'].values

In [11]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).ravel() 

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, shuffle=False)


In [ ]:
# Crea il modello
model = SVR(kernel='rbf', C=100, epsilon=0.1)

# Usa `parallel_backend` per approfittare di più core, se vuoi ottimizzare la velocità
with parallel_backend('loky', n_jobs=-1):
    # Creiamo un oggetto tqdm per la barra di avanzamento
    progress_bar = tqdm(total=len(X_train), desc="Training Progress", position=0, leave=True)
    
    # Addestriamo il modello, aggiornando la barra di avanzamento
    model.fit(X_train, y_train)

    # Aggiorniamo la barra a fine allenamento
    progress_bar.update(len(X_train))
    progress_bar.close()


/home/paolo/miniconda3/envs/myenv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:71: FutureWarning: Class parallel_backend is deprecated; deprecated in 1.5 to be removed in 1.7. Use joblib.{} instead.
  warnings.warn(msg, category=FutureWarning)
Training Progress:   0%|          | 0/2457596 [00:00<?, ?it/s]

In [ ]:
y_pred_scaled = model.predict(X_test)

# Invertiamo la trasformazione per riportare i dati alla scala originale
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).ravel()
y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1)).ravel()

# Calcoliamo l'errore
mse = mean_squared_error(y_test_original, y_pred)
print(f'MSE: {mse:.2f}')

# Grafico delle previsioni vs dati reali
plt.figure(figsize=(10,5))
plt.plot(y_test_original[:500], label='Valori reali', linestyle='dashed')
plt.plot(y_pred[:500], label='Predizioni SVR')
plt.legend()
plt.xlabel('Indice')
plt.ylabel('Valore')
plt.title('Confronto tra valori reali e predetti')
plt.show()